## SPI-Hydration _ D2O shots _ analysis
#### Find the single D2O shots and comapre it to the TOF electron spectromter data

#### Importing Packages

In [1]:
# for importing files
import sys, glob
import os, re
# for the plot stuff
%matplotlib widget
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
#from matplotlib import style
## change font
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams.update({'font.size': 14}) # x,y tick font
matplotlib.rcParams.update({'font.weight': 'bold'}) # x,y tick font bold
# For Intractive plotting
import ipywidgets as widgets
from ipywidgets import interactive, FloatSlider, IntSlider
# numpy package
import numpy as np
## For HDF file format
# import tables
import h5py
import pandas as pd 

#### Run number 

In [2]:
run_number = 54
buffer = 'D2O'

#### Importing HDF(.h5) file using h5py (http://docs.h5py.org/en/stable/)

In [3]:
path_name = '/Volumes/JCPK_BACKUP/PDF_Data/LCLS_SPI-Hydration/'+buffer+'_shots'
## https://docs.python.org/release/2.5.2/lib/typesseq-strings.html - %04d
filename_ol2 = (path_name + '/amolq4015_r%04d_ol2.h5' % run_number)
filename_ol3 = (path_name + '/amolq4015_r%04d_ol3.h5' % run_number)
### h5file = h5py.File(filename,'r')  #file name as required
h5file_ol2 = h5py.File(filename_ol2,'r') # ToF Data 
h5file_ol3 = h5py.File(filename_ol3, 'r') # pnccd Detector Data

#### Pulse Energy

In [4]:
detector_4 = h5file_ol3['entry_1/detector_4/data']  #shows how to access the data, change as required
#
mean = np.mean(detector_4)
#
plt.close(fig=1)
fig = plt.figure(num=1, figsize=(10, 5), dpi=80, facecolor='lightgrey', edgecolor='k')
plt.plot(detector_4, 'g.', label = 'Pulse energy', zorder=1)
plt.hlines(mean, 0, len(detector_4), colors='r', linestyles='dashed', linewidth=3.0, label = 'Mean energy', zorder=2)
plt.title("Mean energy = "+str(round(mean,2))+" mJ")
plt.xlabel('Shot number')
plt.ylabel('Pulse energy (mJ)')
#plt.ylim((37,39))
plt.grid(True)
plt.legend(loc = 4)
plt.tight_layout()
# plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
plt.show()

FigureCanvasNbAgg()

#### Detector 1 + ToF

In [5]:
det1_data = h5file_ol3['entry_1/detector_1/data']  #shows how to access the data, change as required
#
tof_data_y = h5file_ol2['entry_1/detector_2/data_corrected_y']
tof_data_x = h5file_ol2['entry_1/detector_2/data_corrected_x']
len(det1_data), len(tof_data_x), len(tof_data_y)

(702, 702, 702)

In [6]:
plt.close(fig=2)
def f(j):
    fig = plt.figure(num=2, figsize=(13, 6), dpi=80, facecolor='lightgrey', edgecolor='k')
    plt.clf()
    plt.subplot(1,2,1)
    plt.imshow(det1_data[j,:,:].astype(np.float32), cmap='jet')
    plt.clim(vmin=0,vmax=300)
    plt.title("All shots, shot# - "+str(j))
    plt.xlabel('pixels')
    plt.ylabel('pixels')
    plt.colorbar(fraction=0.0435, pad=0.04)
    plt.tight_layout()
    plt.show()
    #
    plt.subplot(1,2,2)
    plt.plot(tof_data_x[j,:], tof_data_y[j,:], 'g.--')
    plt.xlim((-0.1, 30))
    plt.title("ToF spectrum , shot# - "+str(j))
    plt.xlabel('m/q')
    plt.ylabel('Readout voltage (V)')
    plt.grid(True)
    plt.tight_layout()
    # plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
    plt.show()
##
a = widgets.BoundedIntText(min=0, max=len(det1_data)-1, step=1, description='Shot # :', disabled=False)
b = widgets.IntSlider(min=0, max=len(det1_data)-1, step=1, continous_update = False, orientation='horizontal', description='shot number')
# interactive_plot_c = interactive(f_c, k=widgets.IntSlider(min=0, max=len(idx_good_shots)-1, step=1, continous_update = False, orientation='horizontal', description='shot number'))
interactive_plot = interactive(f, j = a)
display(b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))
interactive_plot.children[-1].layout.height = '550px'
interactive_plot
# interactive_plot = interactive(f, j=widgets.IntSlider(min=0, max=len(det1_data)-1, step=1, continous_update = False, orientation='horizontal', description='shot number'))
# # interactive_plot.children[-1].layout.height = '550px'
# interactive_plot

IntSlider(value=0, description='shot number', max=701)

interactive(children=(BoundedIntText(value=0, description='Shot # :', max=701), Output(layout=Layout(height='5…

#### Limit in intensity

In [7]:
result_pix = h5file_ol3['entry_1/result_1/hitscore_litpixel']  #shows how to access the data, change as required
#
# pix_lim = 5000
pix_lim = 10000
#
plt.close(fig=3)
fig = plt.figure(num=3, figsize=(10, 6), dpi=80, facecolor='lightgrey', edgecolor='k')
plt.plot(result_pix, 'go', label = 'pixel intensity', zorder=1)
plt.hlines(pix_lim, 0, len(result_pix), colors='r', linestyles='dashed', linewidth=3.0, label = 'Limit', zorder=2)
plt.title("hitscore_litpixel")
plt.xlabel('Shot number')
plt.ylabel('Pixel value')
#plt.ylim((37,39))
plt.grid(True)
plt.legend()
plt.tight_layout()
# plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
plt.show()

FigureCanvasNbAgg()

In [8]:
pix_idx = (np.where(result_pix[:] > pix_lim))
idx = pix_idx[0]
len(idx)

82

#### Plot of det 1 to check for double shot

In [9]:
plt.close(fig=4)
def f_g(k):
    fig = plt.figure(num=4, figsize=(13, 6), dpi=80, facecolor='lightgrey', edgecolor='k')
    plt.clf()
    plt.suptitle('Check for double shots, Run # '+str(run_number), fontweight='bold')
    plt.subplot(1,2,1)
    plt.imshow(det1_data[idx[k],:,:].astype(np.float32), cmap='jet')
    plt.clim(vmin=0,vmax=300)
    plt.title("D2O shots #"+str(idx[k]))
    plt.xlabel('pixels')
    plt.ylabel('pixels')
    plt.colorbar()
    plt.tight_layout()
    # plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
    plt.show()
    #
    plt.subplot(1,2,2)
    plt.plot(tof_data_x[idx[k],:], tof_data_y[idx[k],:], 'g.--')
    plt.xlim((-0.05, 5))
    plt.title("ToF spectrum , shot# - "+str(idx[k]))
    plt.xlabel('m/q')
    plt.ylabel('Readout voltage (V)')
    plt.grid(True)
    plt.tight_layout()
    # plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
    plt.show()
##
a = widgets.BoundedIntText(min=0, max=len(idx)-1, step=1, description='Shot # :', disabled=False)
b = widgets.IntSlider(min=0, max=len(idx)-1, step=1, continous_update = False, orientation='horizontal', description='shot number')
# interactive_plot_c = interactive(f_c, k=widgets.IntSlider(min=0, max=len(idx_good_shots)-1, step=1, continous_update = False, orientation='horizontal', description='shot number'))
interactive_plot_g = interactive(f_g, k = a)
display(b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))
interactive_plot_g.children[-1].layout.height = '550px'
interactive_plot_g
# interactive_plot_g = interactive(f_g, k=widgets.IntSlider(min=0, max=len(idx)-1, step=1, continous_update = False, orientation='horizontal', description='shot number'))
# interactive_plot_g.children[-1].layout.height = '550px'
# interactive_plot_g

IntSlider(value=0, description='shot number', max=81)

interactive(children=(BoundedIntText(value=0, description='Shot # :', max=81), Output(layout=Layout(height='55…

##### Single D20 shots selected after manually checking all the shots above the threshold

In [10]:
# ## Use for the first test
# ## index of good shots in idx (from above)
# idx_good_shots_idx = np.loadtxt("Results/Index/d2o_single_idx_run%02d.txt" %run_number, dtype = 'int32', delimiter="\n", unpack=False)
# idx_good_shots = idx[idx_good_shots_idx]
# ## Save shot number for single D2O shots to a text file
# np.savetxt("Results/Index/d2o_single_run%02d.txt" %run_number, idx_good_shots, fmt='%4d', delimiter=' ', newline='\n')
# idx_good_shots[0], len(idx_good_shots)

##### Load the single goos shots indices

In [11]:
idx_good_shots =  np.loadtxt("/Users/jayanathkoliyadu/OneDrive - Universidade de Lisboa/ACADEMICS/PDF/RESEARCH/Data_Analysis/LCLS_SPI-Hydration_2017/Analysis/Results/Index/d2o_single_run%02d.txt" %run_number, dtype = 'int32', delimiter="\n", unpack=False)
##
idx_good_shots[0], len(idx_good_shots)

(1, 69)

#### Plot detector + ToF (good shots only !)

In [12]:
plt.close(fig=5)
# combined - c
def f_c(k):
    fig = plt.figure(num=5, figsize=(13, 6), dpi=80, facecolor='lightgrey', edgecolor='k')
    plt.suptitle('Run # '+str(run_number), fontweight='bold')
    plt.clf()
    plt.suptitle('Run # '+str(run_number) + ' - '+ buffer, fontweight='bold')
    plt.subplot(1,2,1)
    plt.imshow(det1_data[idx_good_shots[k],:,:].astype(np.float32), cmap='jet')
    plt.clim(vmin=0,vmax=300)
    plt.title("Single D2O shot # - "+str(idx_good_shots[k]))
    plt.xlabel('pixels')
    plt.ylabel('pixels')
    plt.colorbar(fraction=0.0435, pad=0.04)
    plt.tight_layout()
    # plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
    plt.show()
    plt.subplot(1,2,2)
    plt.plot(tof_data_x[idx_good_shots[k],:], tof_data_y[idx_good_shots[k],:], 'g.--')
    plt.xlim((-0.1,5))
    plt.title("ToF spectrum shot # - "+str(idx_good_shots[k]))
    plt.xlabel('m/q')
    plt.ylabel('Readout voltage (V)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
#     plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
##
a = widgets.BoundedIntText(min=0, max=len(idx_good_shots)-1, step=1, description='Shot # :', disabled=False)
b = widgets.IntSlider(min=0, max=len(idx_good_shots)-1, step=1, continous_update = False, orientation='horizontal', description='shot number')
# interactive_plot_c = interactive(f_c, k=widgets.IntSlider(min=0, max=len(idx_good_shots)-1, step=1, continous_update = False, orientation='horizontal', description='shot number'))
interactive_plot_c = interactive(f_c, k = a)
display(b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))
interactive_plot_c.children[-1].layout.height = '550px'
interactive_plot_c

IntSlider(value=0, description='shot number', max=68)

interactive(children=(BoundedIntText(value=0, description='Shot # :', max=68), Output(layout=Layout(height='55…

#### H+, D+ Peak Analysis

In [13]:
## From conf_preproc_at_slac.py
## H+
h_peak_start = 0.90
h_peak_end = 1.1
## D+
d_peak_start = 1.8
d_peak_end = 2.1
#
h_peak_sum  = np.zeros((len(idx_good_shots)))
d_peak_sum  = np.zeros((len(idx_good_shots)))
#
for l in range(0,len(idx_good_shots),1):
    ## H+ peak
    h_peak_index = np.where((tof_data_x[idx_good_shots[l],:] >= h_peak_start) & (tof_data_x[idx_good_shots[l],:] <= h_peak_end))
    h_peak = tof_data_y[idx_good_shots[l], np.min(h_peak_index):np.max(h_peak_index)]
    h_peak_sum[l] = np.sum(h_peak)
    # ## D+ peak
    d_peak_index = np.where((tof_data_x[idx_good_shots[l],:] >= d_peak_start) & (tof_data_x[idx_good_shots[l],:] <= d_peak_end))
    d_peak = tof_data_y[idx_good_shots[l], np.min(d_peak_index):np.max(d_peak_index)]
    d_peak_sum[l] = np.sum(d_peak)

In [14]:
## number fraction D+/H+ 
d_h = d_peak_sum/h_peak_sum
mean_dh = np.mean(d_h)
## Save number fraction for single D2O shots to a text file
# np.savetxt("Results/num_fraction_run%02d.txt" %run_number, d_h, fmt='%4f', delimiter=' ', newline='\n')
#
plt.close(fig=6)
fig = plt.figure(num=6, figsize=(10, 6), dpi=80, facecolor='lightgrey', edgecolor='k')
plt.plot(d_h, 'go', label = 'D+/H+', zorder=1)
plt.hlines(mean_dh, 0, len(d_h), colors='r', linestyles='dotted', linewidth=3.0, label = 'Mean = '+str(round(mean_dh,2)), zorder=2)
plt.title('Number fraction D+/H+, Run #'+ str(run_number) + ' - '+ buffer, fontweight='bold')
plt.xlabel('Shot number')
plt.ylabel('Number fraction D+/H+')
# plt.ylim((0,4))
plt.grid(True)
plt.legend()
plt.tight_layout()
# plt.savefig('Jitter+Drift.png', bbox_inches = 'tight')
plt.show()

FigureCanvasNbAgg()

#### Save analyzed results

In [15]:
# df = pd.DataFrame({'run_number': run_number, 'buffer': buffer, 'idx_good_shots': idx_good_shots, 'number_fraction': d_h, 'mean_d/h': mean_dh})
# ## saving the dataframe 
# df.to_csv('Results/analysis_'+buffer+'_run#%02d.csv' %run_number) 

##### save as HDF

In [16]:
# hf = h5py.File('Results/analysis_'+buffer+'_run#%02d.h5' %run_number, 'w')
# hf.create_dataset('run_number', data = run_number)
# hf.create_dataset('buffer', data = buffer)
# hf.create_dataset('idx_good_shots', data=idx_good_shots)
# hf.create_dataset('pulse_energy', data = np.asarray(detector_4)[idx_good_shots])
# hf.create_dataset('diff_pat', data = det1_data[np.sort(idx_good_shots),:,:])
# hf.create_dataset('tof_x', data = tof_data_x[np.sort(idx_good_shots),:])
# hf.create_dataset('tof_y', data = tof_data_y[np.sort(idx_good_shots),:])
# hf.create_dataset('number_fraction', data=d_h)
# hf.create_dataset('mean_number_fraction', data=mean_dh)
# hf.close()